# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Hughes  ->  A. Hughes  |  ['A. Hughes']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
N. Storm  ->  N. Storm  |  ['N. Storm']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
Arxiv has 37 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.11690


/tmp/ipykernel_2153/3009462391.py:49: LatexWarning: 2308.11690 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2308.11888


extracting tarball to tmp_2308.11888...

 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2308.12092


extracting tarball to tmp_2308.12092... done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2308.12092/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


Found 107 bibliographic references in tmp_2308.12092/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.11888-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.11888) | **First measurement of the Mg II forest correlation function in the Epoch  of Reionization**  |
|| S. S. Tie, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2023-08-24*|
|*Comments*| *Submitted to MNRAS; 23 pages, 21 figures, 1 table*|
|**Abstract**| In the process of producing the roughly three ionizing photons per atom required to reionize the IGM, the same massive stars explode and eject metals into their surroundings. While the overly sensitive Lya transition makes Gunn-Peterson absorption of background quasar light an ineffective probe of reionization at z > 6, strong low-ionization transitions like the MgII doublet will give rise to a detectable "metal-line forest", if metals pollute the neutral IGM. We measure the auto-correlation of the MgII forest transmission using a sample of ten ground based z >= 6.80 quasar spectra probing the redshift range 5.96 < z_MgII < 7.42 (z_MgII,median = 6.47). The correlation function exhibits strong small-scale clustering and a pronounced peak at the doublet velocity (768 km/s) arising from strong absorbers in the CGM of galaxies. After these strong absorbers are identified and masked the signal is consistent with noise. Our measurements are compared to a suite of models generated by combining a large hydrodynamical simulation with a semi-numerical reionization topology, assuming a simple uniform enrichment model. We obtain a 95% credibility upper limit of [Mg/H] < -3.73 at z_MgII,median = 6.47, assuming uninformative priors on [Mg/H] and the IGM neutral fraction x_HI. Splitting the data into low-z (5.96 < z_MgII < 6.47; z_MgII,median = 6.235) and high-z (6.47 < z_MgII < 7.42; z_MgII,median = 6.72) subsamples again yields null-detections and 95% upper limits of [Mg/H] < -3.75 and [Mg/H] < -3.45, respectively. These first measurements set the stage for an approved JWST Cycle 2 program (GO 3526) targeting a similar number of quasars that will be an order of magnitude more sensitive, making the Mgii forest an emerging powerful tool to deliver precision constraints on the reionization and enrichment history of the Universe. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.12092-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.12092) | **Observational constraints on the origin of the elements. VII. NLTE  analysis of Y II lines in spectra of cool stars and implications for Y as a  Galactic chemical clock**  |
|| <mark>N. Storm</mark>, <mark>M. Bergemann</mark> |
|*Appeared on*| *2023-08-24*|
|*Comments*| *12 pages, 10 figures, accepted by MNRAS*|
|**Abstract**| Yttrium (Y), a key s-process element, is commonly used in nucleosynthesis studies and as a Galactic chemical clock when combined with magnesium (Mg). We study the applicability of the previously assumed LTE line formation assumption in Y abundance studies of main-sequence and red giant stars, and probe the impact of NLTE effects on the [Y/Mg] ratio, a proposed stellar age indicator. We derive stellar parameters, ages, and NLTE abundances of Fe, Mg, and Y for 48 solar analogue stars from high-resolution spectra acquired within the Gaia-ESO survey. For Y, we present a new NLTE atomic model. We determine a solar NLTE abundance of A(Y)$_{\rm NLTE}=2.12\pm0.04$ dex, $0.04$ dex higher than LTE. NLTE effects on Y abundance are modest for optical Y II lines, which are frequently used in Sun-like stars diagnostics. NLTE has a small impact on the [Y/Mg] ratio in such stars. For metal-poor red giants, NLTE effects on Y II lines are substantial, potentially exceeding $+0.5$ dex. For the Gaia/4MOST/WEAVE benchmark star, HD 122563, we find the NLTE abundance ratio of [Y/Fe]$_{\rm NLTE}=-0.55\pm0.04$ dex with consistent abundances obtained from different Y II lines. NLTE has a differential effect on Y abundance diagnostics in late-type stars. They notably affect Y II lines in red giants and very metal-poor stars, which are typical Galactic enrichment tracers of neutron-capture elements. For main-sequence stars, NLTE effects on optical diagnostic Y II lines remain minimal across metallicities. This affirms the [Y/Mg] ratio's reliability as a cosmochronometer for Sun-like stars. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.11690-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.11690) | **The case for studying other planetary magnetospheres and atmospheres in  Heliophysics**  |
|| I. J. Cohen, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2023-08-24*|
|*Comments*| **|
|**Abstract**| Heliophysics is the field that ``studies the nature of the Sun, and how it influences the very nature of space - and, in turn, the atmospheres of planetary bodies and the technology that exists there.'' However, NASA's Heliophysics Division tends to limit study of planetary magnetospheres and atmospheres to only those of Earth. This leaves exploration and understanding of space plasma physics at other worlds to the purview of the Planetary Science and Astrophysics Divisions. This is detrimental to the study of space plasma physics in general since, although some cross-divisional funding opportunities do exist, vital elements of space plasma physics can be best addressed by extending the expertise of Heliophysics scientists to other stellar and planetary magnetospheres. However, the diverse worlds within the solar system provide crucial environmental conditions that are not replicated at Earth but can provide deep insight into fundamental space plasma physics processes. Studying planetary systems with Heliophysics objectives, comprehensive instrumentation, and new grant opportunities for analysis and modeling would enable a novel understanding of fundamental and universal processes of space plasma physics. As such, the Heliophysics community should be prepared to consider, prioritize, and fund dedicated Heliophysics efforts to planetary targets to specifically study space physics and aeronomy objectives. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.11888.md
    + _build/html/tmp_2308.11888/./figures/spec_J0313-1806.png
    + _build/html/tmp_2308.11888/./figures/spec_J1342+0928.png
    + _build/html/tmp_2308.11888/./figures/spec_J1007+2115.png
    + _build/html/tmp_2308.11888/./figures/allz_corr_func_data.png
    + _build/html/tmp_2308.11888/./figures/allz_corner_plot.png
    + _build/html/tmp_2308.11888/./figures/masked4_J0313-1806.png
    + _build/html/tmp_2308.11888/./figures/masked7_J1342+0928.png
    + _build/html/tmp_2308.11888/./figures/masked8_J1007+2115.png
exported in  _build/html/2308.12092.md
    + _build/html/tmp_2308.12092/./figures/y_loggteff_low_v2.png
    + _build/html/tmp_2308.12092/./figures/y_loggteff_medium_v2.png
    + _build/html/tmp_2308.12092/./figures/yh_elow_v3.png
    + _build/html/tmp_2308.12092/./figures/yfe_ew_v3.png
    + _build/html/tmp_2308.12092/./figures/s4500_m0_departure_coefficients.png
    + _build/html/tmp_2308.12092/./figures/s4500_m3_departure_coefficients.pn

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\actaa}{\rm Acta. Astr.}$
$\newcommand{\apj}{\rm ApJ}$
$\newcommand{\apjl}{\rm ApJL}$
$\newcommand{\apjs}{\rm ApJS}$
$\newcommand{\aj}{\rm AJ}$
$\newcommand{\jcap}{\rm JCAP}$
$\newcommand{\mnras}{\rm MNRAS}$
$\newcommand{\nat}{\rm Nature}$
$\newcommand{\pasj}{\rm PASJ}$
$\newcommand{\pasp}{\rm PASP}$
$\newcommand{\pasa}{\rm PASA}$
$\newcommand{\aap}{\rm AAP}$
$\newcommand{\araa}{\rm ARA\&A}$
$\newcommand{\na}{\rm New Astronomy}$
$\newcommand{\nar}{\rm New Astronomy Review}$
$\newcommand{\prl}{\rm PRL}$
$\newcommand{\prd}{\rm PRD}$
$\newcommand{\apss}{\rm Ap\&SS}$
$\newcommand{\physrep}{\rm Physics Reports}$
$\newcommand{\rmxaa}{\rm RMXAA}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\hcmpc}{h^{-1} \mbox{ cMpc}}$
$\newcommand{\mgii}{\ion{Mg}{\uppercase{II}}}$
$\newcommand{\qsoa}{J0313-1806}$
$\newcommand{\qsob}{J1342+0928}$
$\newcommand{\qsoc}{J0252-0503}$
$\newcommand{\qsod}{J0038-1527}$</div>



<div id="title">

# First measurement of the $\ion{Mg}{\uppercase{II}}$ forest correlation function in the Epoch of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.11888-b31b1b.svg)](https://arxiv.org/abs/2308.11888)<mark>Appeared on: 2023-08-24</mark> -  _Submitted to MNRAS; 23 pages, 21 figures, 1 table_

</div>
<div id="authors">

S. S. Tie, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** In the process of producing the roughly three ionizing photons per atom required to reionize the IGM, the same massive stars explode and eject metals into their surroundings, enriching the Universe to $Z\sim 10^{-3} Z_{\odot}$ . While the overly sensitive Ly $\alpha$ transition makes Gunn-Peterson absorption of background quasar light an ineffective probe of reionization at $z > 6$ , strong low-ionization transitions like the $\ion{Mg}{II}$ $\lambda 2796,2804$ Å  doublet will give rise to a detectable `metal-line forest', if metals pollute the neutral IGM. We measure the auto-correlation of the $\ion{Mg}{II}$ forest transmission using a sample of ten ground based $z \geq 6.80$ quasar spectra probing the redshift range $5.96 < z_{\rm \ion{Mg}{II}} < 7.42$ ( $z_{\rm \ion{Mg}{II}, median} = 6.47$ ).  The correlation function exhibits strong small-scale clustering and a pronounced peak at the doublet velocity  ( $\Delta v = 768 {\rm km s^{-1}}$ ) arisingfrom strong absorbers in the CGM of galaxies. After these strong absorbers are identified and masked the signal is consistent with noise. Our measurements are compared to a suite of models generated by combining a large hydrodynamical simulation with a semi-numerical reionization topology, assuming a simple uniform enrichment model.  We obtain a 95 \% credibility upper limit of $[{\rm Mg/H}] <-3.73$ at $z_{\rm \ion{Mg}{II},median} = 6.47$ , assuming uninformative priors on [ Mg/H ] and the IGM neutral fraction $x_{\rm{\ion{H}{I}}}$ .Splitting the data into low- $z$ ( $5.96 < z_{\rm \ion{Mg}{II}} < 6.47$ ; $z_{\rm \ion{Mg}{II},median} = 6.235$ ) and high- $z$ ( $6.47 < z_{\rm \ion{Mg}{II}} < 7.42$ ; $z_{\rm \ion{Mg}{II},median} = 6.72$ ) subsamples again yields null-detections and  95 \% upper limits of $[{\rm Mg/H}] <-3.75$ and $[{\rm Mg/H}] <-3.45$ , respectively.  These first measurements set the stage for an approved JWST Cycle 2 program (GO 3526) targeting a similar number of quasars that will be an order of magnitude more sensitive, making the $\ion{Mg}{II}$ forest an emerging powerful tool to deliver precision constraints on the reionization and enrichment history of the Universe.

</div>

<div id="div_fig1">

<img src="tmp_2308.11888/./figures/spec_J0313-1806.png" alt="Fig3.1" width="33%"/><img src="tmp_2308.11888/./figures/spec_J1342+0928.png" alt="Fig3.2" width="33%"/><img src="tmp_2308.11888/./figures/spec_J1007+2115.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Three example spectra from our dataset, J0313$-$1806 (top), J1342$+$0928 (middle), and J1007$+$2115 (bottom). In the top panels of each quasar, the red line  is the fitted quasar continuum, the faint black line is the spectral noise, and the shaded regions indicate masks obtained from \texttt{PypeIt} reduction and from visually-detected strong absorbers, which are applied before continuum fitting. The bottom panel of each quasar shows the continuum-normalized spectrum. Also shown is the telluric spectrum in blue. The multi-colored shaded regions indicate masks due to \texttt{PypeIt} reduction (if available, in green), proximity to quasars and where $z > z_{\rm{QSO}}$(gray), low SNR due to telluric absorption (blue), and detected CGM absorbers (red; see \S 4), where we use the unshaded regions for the correlation function measurement and inference. The vertical dashed lines indicate the quasar redshift. (*specj0313*)

</div>
<div id="div_fig2">

<img src="tmp_2308.11888/./figures/allz_corr_func_data.png" alt="Fig13.1" width="50%"/><img src="tmp_2308.11888/./figures/allz_corner_plot.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** Results from our MCMC analyses in the all-$z$ redshift bin ($z_{\rm{$\ion${Mg}{II},med}} = 6.469$). _Left:_ Correlation function measured from our dataset compared with 200 random draws (blue lines) and the mean inferred model (red line) from the MCMC posterior distribution. The black points are measurements from our dataset, where we have masked the gray points before running the MCMC sampling (see Appendix A). The error bars are computed from the diagonal elements of the covariance matrix of the inferred model. _Right:_ Corner plot from MCMC sampling of the posterior distribution. The 95\% C.L. upper limit on [Mg/H] is $-3.73$. (*allz-mcmc*)

</div>
<div id="div_fig3">

<img src="tmp_2308.11888/./figures/masked4_J0313-1806.png" alt="Fig8.1" width="33%"/><img src="tmp_2308.11888/./figures/masked7_J1342+0928.png" alt="Fig8.2" width="33%"/><img src="tmp_2308.11888/./figures/masked8_J1007+2115.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** Identified and subsequently masked CGM absorbers from J0313$-$1806 (top), J1342$+$0928 (middle), and J1007$+$2115 (bottom), where we have cut off regions of the spectra within the quasar proximity zone and beyond the quasar redshift. For each quasar, the top panel shows the continuum-normalized spectrum and the bottom panel shows the $\chi$ significance spectrum. The green and magenta horizontal lines indicate the thresholds for the $1 - \rm{F}$ and $\chi$ cuts, respectively (note that we additionally mask pixels $\pm 300$ km/s around pixels with $\chi > 3$). Pixels that are masked by the respective cuts are colored similarly. Gray shaded regions are regions of telluric absorptions. Our masking procedure results in a number of weak and strong absorber candidates, while recovering known strong absorbers. (*masked-all-qso*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.11888"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\vmic}{\xi_{\rm t}}$
$\newcommand{\vmac}{V_{\rm mac}}$
$\newcommand{\EW}{W_{\lambda}}$
$\newcommand{\mA}{{\rm mÅ}}$
$\newcommand{\Elow}{E_{\rm low}}$
$\newcommand{\Eup}{E_{\rm up}}$
$\newcommand{\SH}{S\!_{\rm H}}$
$\newcommand{\footnoterule}$
$\newcommand{\footnoterule}$</div>



<div id="title">

# Observational constraints on the origin of the elements. VII. NLTE analysis of Y II lines in spectra of cool stars and implications for Y as a Galactic chemical clock

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.12092-b31b1b.svg)](https://arxiv.org/abs/2308.12092)<mark>Appeared on: 2023-08-24</mark> -  _12 pages, 10 figures, accepted by MNRAS_

</div>
<div id="authors">

<mark>N. Storm</mark>, <mark>M. Bergemann</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ Yttrium (Y), a key s-process element, is commonly used in nucleosynthesis studies and as a Galactic chemical clock when combined with magnesium (Mg). We study the applicability of the previously assumed LTE line formation assumption in Y abundance studies of main-sequence and red giant stars, and probe the impact of NLTE effects on the [ Y/Mg ] ratio, a proposed stellar age indicator. We derive stellar parameters, ages, and NLTE abundances of Fe, Mg, and Y for 48 solar analogue stars from high-resolution  spectra acquired within the Gaia-ESO survey. For Y, we present a new NLTE atomic model. We determine a solar NLTE abundance of A(Y) $_{\rm NLTE}=2.12\pm0.04$ dex, $0.04$ dex higher than LTE. NLTE effects on Y abundance are modest for optical Y II lines, which are frequently used in Sun-like stars diagnostics. NLTE has a small impact on the [ Y/Mg ] ratio in such stars. For metal-poor red giants, NLTE effects on Y II lines are substantial, potentially exceeding $+0.5$ dex. For the Gaia/4MOST/WEAVE benchmark star, HD 122563, we find the NLTE abundance ratio of [ Y/Fe ] $_{\rm NLTE}=-0.55\pm0.04$ dex with consistent abundances obtained from different Y II lines. NLTE has a differential effect on Y abundance diagnostics in late-type stars. They notably affect Y II lines in red giants and very metal-poor stars, which are typical Galactic enrichment tracers of neutron-capture elements. For main-sequence stars, NLTE effects on optical diagnostic Y II lines remain minimal across metallicities. This affirms the [ Y/Mg ] ratio's reliability as a cosmochronometer for Sun-like stars.

</div>

<div id="div_fig1">

<img src="tmp_2308.12092/./figures/y_loggteff_low_v2.png" alt="Fig9.1" width="50%"/><img src="tmp_2308.12092/./figures/y_loggteff_medium_v2.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** Average NLTE corrections for the low-EP (multiplets 7,8,6) and medium-EP (multiplets 21, 23) Y II lines plotted against metallicity [Fe/H] of the model atmospheres. We adopt $\vmic = 2$ km/s and $\vmic = 1$ km/s for the models with log(g)$ \leq 3.5$ dex and log(g)$ > 3.5$ dex respectively. (*fig:nltecor*)

</div>
<div id="div_fig2">

<img src="tmp_2308.12092/./figures/yh_elow_v3.png" alt="Fig5.1" width="50%"/><img src="tmp_2308.12092/./figures/yfe_ew_v3.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Y abundances for HD 122563 ([Y/H] on the left and [Y/Fe] on the right) plotted as a function of lower energy state EP or EW. The red and black points represent respectively fitted LTE and NLTE abundances. Best-fit lines going through the low-EP line 3832.89 are also plotted in relevant colours. (*fig:yfe_yh_elow*)

</div>
<div id="div_fig3">

<img src="tmp_2308.12092/./figures/s4500_m0_departure_coefficients.png" alt="Fig10.1" width="25%"/><img src="tmp_2308.12092/./figures/s4500_m3_departure_coefficients.png" alt="Fig10.2" width="25%"/><img src="tmp_2308.12092/./figures/p6000_m0_departure_coefficients.png" alt="Fig10.3" width="25%"/><img src="tmp_2308.12092/./figures/p6000_m3_departure_coefficients.png" alt="Fig10.4" width="25%"/>

**Figure 10. -** {Departure coefficients as a function of model atmosphere's optical depth for a red giant model on top ($\teff = 4500$ K, $\logg = 2.0$ dex for [Fe/H] = -3 and 0, respectively left and right plots) and a main-sequence on the bottom ($\teff = 4500$ K, $\logg = 2.0$ dex for [Fe/H] = -3 and 0, respectively left and right plots). Low-EP line (3832 Å) levels (a $^3$$D_3$ and z $^3$$F_3$*) are depicted in black and medium-EP line (4883 Å) levels (a $^3$$F_4$ and z $^3$$D_3$*) in red, with solid and dashed lines representing lower and upper levels, respectively. Vertical lines indicate line core at $\log \tau = 0$ for both low-EP (black) and medium-EP (red) lines.} (*fig:depcoef_rg*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.12092"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

251  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
